In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

In [ ]:
df = pd.read_csv('../data/broadband_subscriptions.csv', skiprows=4)

In [ ]:
df.describe()

In [ ]:
columns_to_drop = np.array(range(1960,1998))
df.drop(columns=columns_to_drop.astype('str'), inplace=True)

In [ ]:
df.head()

In [ ]:
countries = pd.read_csv('../data/country-codes_csv.csv')

Not all country codes are ISO codes. We will have to fix it.

In [ ]:
df = df.merge(countries['ISO3166-1-Alpha-3'], how='left', left_on='Country Code', right_on='ISO3166-1-Alpha-3')

In [ ]:
import difflib
countries = pd.read_csv('../data/country-codes_csv.csv')
def get_country_iso(name):
    p = difflib.get_close_matches(name, countries['official_name_en'].dropna().unique(), n=1, cutoff=0.5)
    if len(p)>0:
        return countries[countries['official_name_en']==p[0]]['ISO3166-1-Alpha-3'].to_list()[0]
    else:
        return ""

In [ ]:
df.rename(columns={'ISO3166-1-Alpha-3': 'country_iso'}, inplace=True)
df[df['country_iso']=='']['country_iso'] = [get_country_iso(x) for x in df[df['country_iso']=='']['Country Name']]
df[df['country_iso'].isna()][['Country Code', 'Country Name']]

None of the missing values correspond to a country. So, we can drop the rows.

In [ ]:
df.drop(index=df[df['country_iso'].isna()].index, inplace=True)
df[df['country_iso'].isna()]['Country Name'].unique()

In [ ]:
df[df['Country Code']!=df['country_iso']]

In [ ]:
df.drop(columns=['Country Code'], inplace=True)
columns_to_melt = np.array(range(1998,2020)).astype('str')

In [ ]:
df_new = df[['country_iso', *columns_to_melt]]
df_new

In [ ]:
df_new = df_new.melt(id_vars=["country_iso"],
        var_name="year", 
        value_name="broadband_subscriptions_per100")
df_new

In [ ]:
pd.set_option('display.max_rows', None)
df_new = df_new.sort_values(['country_iso', 'year'])

In [ ]:
df_new.groupby(['country_iso', 'year']).count().max()

Now, we will drop the rows containing no value for the feature:

In [ ]:
df_new.drop(index=df_new[df_new['broadband_subscriptions_per100'].isna()].index, inplace=True)

In [ ]:
df_new.head()

In [ ]:
df_new.to_csv('../data/converted/broadband_subscriptions.csv', index=False)

In [ ]:
df_saved = pd.read_csv('../data/converted/broadband_subscriptions.csv')
df_saved.head()

In [ ]:
df_saved.count()